# This notebook calculates the winrate of a team

In [214]:
import pandas as pd
import numpy as np
pd.options.display.max_columns = None
import warnings
warnings.filterwarnings('ignore')

In [185]:
# We start of by importing the pivot tables for synergy-winrate and counter-winrate
synergy_pivot_2000_3000 = pd.read_pickle("../data/2000_3000_synergy_pivot.pkl")
counter_pivot_2000_3000 = pd.read_pickle("../data/2000_3000_counter_pivot.pkl")
synergy_pivot_3000_4000 = pd.read_pickle("../data/3000_4000_synergy_pivot.pkl")
counter_pivot_3000_4000 = pd.read_pickle("../data/3000_4000_counter_pivot.pkl")
synergy_pivot_no_limit = pd.read_pickle("../data/no_limit_synergy_pivot.pkl")
counter_pivot_no_limit = pd.read_pickle("../data/no_limit_counter_pivot.pkl")

# We import the train data
train_2000_3000 = pd.read_pickle("../data/clean_train_2000_3000.pkl")
train_3000_4000 = pd.read_pickle("../data/clean_train_3000_4000.pkl")
train_no_limit = pd.read_pickle("../data/clean_train_no_limit.pkl")

# We import the test data
test_2000_3000 = pd.read_pickle("../data/clean_test_2000_3000.pkl")
test_3000_4000 = pd.read_pickle("../data/clean_test_3000_4000.pkl")
test_no_limit = pd.read_pickle("../data/clean_test_no_limit.pkl")


### 1. We start off by writing a function to lookup the value of a pair-winrate

In [172]:
def winrate_selector_synergy(hero_pair_tuple, synergy_pivot):
    
    # Locate the hero chosen on the team
    hero_matchup_rows = synergy_pivot.loc[hero_pair_tuple[0]]
    
    # We select the given hero through the index of the list
    winrate = hero_matchup_rows[hero_pair_tuple[1]]
    
    # We return the winrate
    return winrate

In [173]:
def winrate_selector_counter(hero_pair_tuple, counter_pivot):
    
    # Locate the hero chosen on the team
    hero_matchup_rows = counter_pivot.loc[hero_pair_tuple[0]]
    
    # We select the given hero through the index of the list
    winrate = hero_matchup_rows[hero_pair_tuple[1]]
    
    # We return the winrate
    return winrate

### 2. We make a function that calculates the winrate of the tuples in the match

In [174]:
def synergy_winrate_calculator(match_data, player_slots, synergy_pivot):
    for slot1 in player_slots:
        for slot2 in player_slots:
            if slot1 != slot2:
                series = pd.DataFrame(zip(match_data[slot1].values,match_data[slot2].values))
                winrate = series.apply(lambda x: winrate_selector_synergy(x,synergy_pivot), axis=1)
                match_data[f"syn-{slot1}-{slot2}"] = winrate
                print("Synergy ",slot1,slot2)
    return match_data

def counter_winrate_calculator(match_data, player_slots_team, player_slots_opponent, counter_pivot):
    for slot1 in player_slots_team:
        for slot2 in player_slots_opponent:
            series = pd.DataFrame(zip(match_data[slot1].values,match_data[slot2].values))
            winrate = series.apply(lambda x: winrate_selector_counter(x,counter_pivot), axis=1)
            match_data[f"coun-{slot1}-{slot2}"] = winrate
            print("Counter ",slot1,slot2)
    return match_data

In [256]:
def winrate_calculator(match_data, synergy_pivot, counter_pivot):
    
    # Declare variables
    radiant_slots = ["radiant_1",
                     "radiant_2",
                     "radiant_3",
                     "radiant_4",
                     "radiant_5"]
    
    dire_slots = ["dire_1",
                  "dire_2",
                  "dire_3",
                  "dire_4",
                  "dire_5"]
    
    columns = ["match_id",
               "match_seq_num",
               "radiant_win",
               "start_time",
               "duration",
               "avg_mmr",
               "radiant_1",
               "radiant_2",
               "radiant_3",
               "radiant_4",
               "radiant_5",
               "dire_1",
               "dire_2",
               "dire_3",
               "dire_4",
               "dire_5",
               "radiant_synergy_wr",
               "radiant_counter_wr",
               "dire_synergy_wr",
               "dire_counter_wr",
               "rad_syn_mvp",
               "rad_coun_mvp",
               "dire_syn_mvp",
               "dire_coun_mvp"]
    
    winrate_radiant_syn_list = []
    winrate_dire_syn_list = []
    winrate_radiant_coun_list = []
    winrate_dire_coun_list = []
    radiant_winrate = 0
    dire_winrate = 0
    copy_match_data = match_data
    
    # Radiant winrate based on synergy and counter
    radiant_synergy_winrate_df = synergy_winrate_calculator(copy_match_data, radiant_slots, synergy_pivot)
    radiant_counter_winrate_df = counter_winrate_calculator(copy_match_data, radiant_slots, dire_slots,  counter_pivot)
    syn_columns = radiant_synergy_winrate_df.columns[16:36].tolist()
    coun_columns = radiant_counter_winrate_df.columns[36:61].tolist()
    match_data["radiant_synergy_wr"] = copy_match_data[syn_columns].apply(np.mean, axis=1)
    match_data["radiant_counter_wr"] = copy_match_data[coun_columns].apply(np.mean, axis=1)
    match_data = match_data.assign(rad_syn_mvp=np.max(copy_match_data[syn_columns], axis=1))
    match_data = match_data.assign(rad_coun_mvp=np.max(copy_match_data[coun_columns], axis=1))
    
    # Dire winrate based on synergy and counter
    dire_synergy_winrate_df = synergy_winrate_calculator(copy_match_data, dire_slots, synergy_pivot)
    syn_columns = dire_synergy_winrate_df.columns[63:83].tolist()
    match_data["dire_synergy_wr"] = copy_match_data[syn_columns].apply(np.mean, axis=1)
    match_data["dire_counter_wr"] = 1 - match_data["radiant_counter_wr"]
    match_data = match_data.assign(dire_syn_mvp=np.max(copy_match_data[syn_columns], axis=1))
    match_data = match_data.assign(dire_coun_mvp=1-np.min(copy_match_data[coun_columns], axis=1))
    
    return match_data[columns]

In [257]:
train_pred_2000_3000 = winrate_calculator(train_2000_3000, synergy_pivot_2000_3000, counter_pivot_2000_3000)
train_pred_3000_4000 = winrate_calculator(train_3000_4000, synergy_pivot_3000_4000, counter_pivot_3000_4000)
train_pred_no_limit = winrate_calculator(train_no_limit, synergy_pivot_no_limit, counter_pivot_no_limit)

test_pred_2000_3000 = winrate_calculator(test_2000_3000, synergy_pivot_2000_3000, counter_pivot_2000_3000)
test_pred_3000_4000 = winrate_calculator(test_3000_4000, synergy_pivot_3000_4000, counter_pivot_3000_4000)
test_pred_no_limit = winrate_calculator(test_no_limit, synergy_pivot_no_limit, counter_pivot_no_limit)

Synergy  radiant_1 radiant_2
Synergy  radiant_1 radiant_3
Synergy  radiant_1 radiant_4
Synergy  radiant_1 radiant_5
Synergy  radiant_2 radiant_1
Synergy  radiant_2 radiant_3
Synergy  radiant_2 radiant_4
Synergy  radiant_2 radiant_5
Synergy  radiant_3 radiant_1
Synergy  radiant_3 radiant_2
Synergy  radiant_3 radiant_4
Synergy  radiant_3 radiant_5
Synergy  radiant_4 radiant_1
Synergy  radiant_4 radiant_2
Synergy  radiant_4 radiant_3
Synergy  radiant_4 radiant_5
Synergy  radiant_5 radiant_1
Synergy  radiant_5 radiant_2
Synergy  radiant_5 radiant_3
Synergy  radiant_5 radiant_4
Counter  radiant_1 dire_1
Counter  radiant_1 dire_2
Counter  radiant_1 dire_3
Counter  radiant_1 dire_4
Counter  radiant_1 dire_5
Counter  radiant_2 dire_1
Counter  radiant_2 dire_2
Counter  radiant_2 dire_3
Counter  radiant_2 dire_4
Counter  radiant_2 dire_5
Counter  radiant_3 dire_1
Counter  radiant_3 dire_2
Counter  radiant_3 dire_3
Counter  radiant_3 dire_4
Counter  radiant_3 dire_5
Counter  radiant_4 dire_1
Coun

Synergy  dire_3 dire_2
Synergy  dire_3 dire_4
Synergy  dire_3 dire_5
Synergy  dire_4 dire_1
Synergy  dire_4 dire_2
Synergy  dire_4 dire_3
Synergy  dire_4 dire_5
Synergy  dire_5 dire_1
Synergy  dire_5 dire_2
Synergy  dire_5 dire_3
Synergy  dire_5 dire_4
Synergy  radiant_1 radiant_2
Synergy  radiant_1 radiant_3
Synergy  radiant_1 radiant_4
Synergy  radiant_1 radiant_5
Synergy  radiant_2 radiant_1
Synergy  radiant_2 radiant_3
Synergy  radiant_2 radiant_4
Synergy  radiant_2 radiant_5
Synergy  radiant_3 radiant_1
Synergy  radiant_3 radiant_2
Synergy  radiant_3 radiant_4
Synergy  radiant_3 radiant_5
Synergy  radiant_4 radiant_1
Synergy  radiant_4 radiant_2
Synergy  radiant_4 radiant_3
Synergy  radiant_4 radiant_5
Synergy  radiant_5 radiant_1
Synergy  radiant_5 radiant_2
Synergy  radiant_5 radiant_3
Synergy  radiant_5 radiant_4
Counter  radiant_1 dire_1
Counter  radiant_1 dire_2
Counter  radiant_1 dire_3
Counter  radiant_1 dire_4
Counter  radiant_1 dire_5
Counter  radiant_2 dire_1
Counter  ra

In [252]:
test_pred_2000_3000

,match_id,match_seq_num,radiant_win,start_time,duration,avg_mmr,radiant_1,radiant_2,radiant_3,radiant_4,radiant_5,dire_1,dire_2,dire_3,dire_4,dire_5,radiant_synergy_wr,radiant_counter_wr,dire_synergy_wr,dire_counter_wr,rad_syn_mvp,rad_coun_mvp,dire_syn_mvp,dire_coun_mvp
0,5513575804,4623213746,True,1594777959,2511,2139,1,86,23,39,11,108,22,106,8,26,0.482,0.4892,0.501,0.5108,0.56,0.59,0.57,0.63
1,5513576209,4623212187,False,1594777979,2237,2530,45,26,12,29,87,14,17,64,72,99,0.491,0.4980,0.475,0.5020,0.54,0.57,0.53,0.61
2,5513577209,4623218161,True,1594778035,2661,2014,101,44,74,106,15,39,67,31,100,90,0.461,0.4700,0.512,0.5300,0.50,0.53,0.58,0.59
3,5513577815,4623214226,True,1594778061,2438,2812,46,85,25,42,40,45,84,96,48,21,0.495,0.4740,0.541,0.5260,0.59,0.56,0.57,0.60
4,5513578101,4623204258,False,1594778053,1931,2863,84,87,8,71,73,9,95,102,26,48,0.517,0.5024,0.513,0.4976,0.56,0.57,0.54,0.57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,5516010316,4625470045,True,1594857521,2447,2289,41,30,107,70,114,85,58,14,2,32,0.491,0.4920,0.487,0.5080,0.59,0.60,0.57,0.61
9996,5516010413,4625495756,True,1594857511,4019,2036,30,105,41,21,59,71,74,95,49,100,0.502,0.5040,0.495,0.4960,0.55,0.56,0.54,0.60
9997,5516011315,4625472625,False,1594857582,2598,2801,96,126,100,93,50,67,34,104,27,30,0.504,0.4944,0.505,0.5056,0.59,0.56,0.56,0.58
9998,5516011406,4625480100,False,1594857562,3127,2663,126,114,112,129,86,35,79,2,44,26,0.433,0.4724,0.485,0.5276,0.49,0.55,0.54,0.61


**We now create a predictor function to account for the different matchups**

In [271]:
def win_predictor(match_data_pred):
    
    # Team winrates
    match_data_pred["counter_radiant_win"] = np.where(match_data_pred["radiant_counter_wr"] >= 0.5, True, False)
    match_data_pred["synergy_radiant_win"] = np.where(match_data_pred["radiant_synergy_wr"] >= match_data_pred["dire_synergy_wr"], True, False)
    match_data_pred["synergy_precision"] = np.where(match_data_pred["radiant_win"] == match_data_pred["synergy_radiant_win"], True, False)
    match_data_pred["counter_precision"] = np.where(match_data_pred["radiant_win"] == match_data_pred["counter_radiant_win"], True, False)
    
    # MVP winrates
    match_data_pred["synergy_radiant_mvp_win"] = np.where(match_data_pred["rad_syn_mvp"] >= match_data_pred["dire_syn_mvp"], True, False)
    match_data_pred["counter_radiant_mvp_win"] = np.where(match_data_pred["rad_coun_mvp"] >= match_data_pred["dire_coun_mvp"], True, False)
    match_data_pred["synergy_mvp_precision"] = np.where(match_data_pred["radiant_win"] == match_data_pred["synergy_radiant_mvp_win"], True, False)
    match_data_pred["counter_mvp_precision"] = np.where(match_data_pred["radiant_win"] == match_data_pred["counter_radiant_mvp_win"], True, False)
    
    # Team winrates precision
    synergy_counter = match_data_pred["synergy_precision"].value_counts()
    counter_counter = match_data_pred["counter_precision"].value_counts()
    synergy_precision = round(synergy_counter / len(match_data_pred["synergy_precision"]),2)
    counter_precision = round(counter_counter / len(match_data_pred["synergy_precision"]),2)
    
    # MVP winrates precision
    synergy_mvp_counter = match_data_pred["synergy_mvp_precision"].value_counts()
    counter_mvp_counter = match_data_pred["counter_mvp_precision"].value_counts()
    synergy_mvp_precision = round(synergy_mvp_counter / len(match_data_pred["synergy_mvp_precision"]),2)
    counter_mvp_precision = round(counter_mvp_counter / len(match_data_pred["synergy_mvp_precision"]),2)
    
    print("-----------")    
    print(f"The synergy prediction accuracy is {synergy_precision[1]}")
    print(f"The counter prediction accuracy is {counter_precision[1]}")
    print(f"The synergy mvp prediction accuracy is {synergy_mvp_precision[1]}")
    print(f"The counter mvp prediction accuracy is {counter_mvp_precision[1]}")
    print("-----------")
    
    return match_data_pred

In [272]:
# First we test the training sets
train_pred_2000_3000_table = win_predictor(train_pred_2000_3000)
train_pred_3000_4000_table = win_predictor(train_pred_3000_4000)
train_pred_no_limit_table = win_predictor(train_pred_no_limit)

-----------
The synergy prediction accuracy is 0.6
The counter prediction accuracy is 0.6
The synergy mvp prediction accuracy is 0.58
The counter mvp prediction accuracy is 0.59
-----------
-----------
The synergy prediction accuracy is 0.6
The counter prediction accuracy is 0.6
The synergy mvp prediction accuracy is 0.58
The counter mvp prediction accuracy is 0.59
-----------
-----------
The synergy prediction accuracy is 0.6
The counter prediction accuracy is 0.6
The synergy mvp prediction accuracy is 0.58
The counter mvp prediction accuracy is 0.59
-----------


In [273]:
# The prediction accuracy on the test sets are
test_pred_2000_3000_table = win_predictor(test_pred_2000_3000)
test_pred_3000_4000_table = win_predictor(test_pred_3000_4000)
test_pred_no_limit_table = win_predictor(test_pred_no_limit)

-----------
The synergy prediction accuracy is 0.57
The counter prediction accuracy is 0.57
The synergy mvp prediction accuracy is 0.55
The counter mvp prediction accuracy is 0.56
-----------
-----------
The synergy prediction accuracy is 0.56
The counter prediction accuracy is 0.56
The synergy mvp prediction accuracy is 0.55
The counter mvp prediction accuracy is 0.56
-----------
-----------
The synergy prediction accuracy is 0.56
The counter prediction accuracy is 0.57
The synergy mvp prediction accuracy is 0.55
The counter mvp prediction accuracy is 0.55
-----------


In [274]:
train_pred_2000_3000_table

,match_id,match_seq_num,radiant_win,start_time,duration,avg_mmr,radiant_1,radiant_2,radiant_3,radiant_4,radiant_5,dire_1,dire_2,dire_3,dire_4,dire_5,radiant_synergy_wr,radiant_counter_wr,dire_synergy_wr,dire_counter_wr,rad_syn_mvp,rad_coun_mvp,dire_syn_mvp,dire_coun_mvp,counter_radiant_win,synergy_radiant_win,synergy_precision,counter_precision,synergy_radiant_mvp_win,counter_radiant_mvp_win,synergy_mvp_precision,counter_mvp_precision
0,5504961700,4615512732,True,1594245043,1875,2646,30,108,22,8,16,126,39,28,42,68,0.528,0.5116,0.532,0.4884,0.58,0.60,0.60,0.58,True,False,False,True,False,True,False,True
1,5504961819,4615510394,True,1594245052,2055,2321,50,104,44,39,9,41,120,74,105,30,0.507,0.5040,0.489,0.4960,0.55,0.57,0.53,0.55,True,True,True,True,True,True,True,True
2,5504961908,4615513646,True,1594245062,2380,2729,75,30,7,70,108,51,40,10,112,17,0.525,0.5204,0.473,0.4796,0.59,0.62,0.54,0.58,True,True,True,True,True,True,True,True
3,5504961915,4615516612,True,1594245056,2638,2383,68,40,1,82,107,6,64,71,46,25,0.515,0.5148,0.485,0.4852,0.59,0.59,0.57,0.58,True,True,True,True,True,True,True,True
4,5504962018,4615510348,True,1594245071,2116,2275,16,62,44,106,104,120,1,94,35,8,0.487,0.4788,0.501,0.5212,0.53,0.53,0.56,0.59,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99994,5513436411,4623083608,False,1594771154,2584,2100,36,71,32,26,35,11,126,14,94,62,0.510,0.5104,0.468,0.4896,0.57,0.59,0.52,0.55,True,True,False,False,True,True,False,False
99995,5513436513,4623074188,True,1594771155,1923,2416,14,35,104,48,30,47,121,8,23,119,0.534,0.5308,0.474,0.4692,0.57,0.58,0.51,0.53,True,True,True,True,True,True,True,True
99996,5513436514,4623080027,True,1594771159,2398,2682,14,106,104,8,64,90,7,74,67,71,0.515,0.5052,0.504,0.4948,0.55,0.57,0.58,0.56,True,True,True,True,False,True,False,True
99997,5513436705,4623074422,True,1594771169,2052,2198,25,26,14,96,114,73,40,45,93,47,0.451,0.4816,0.499,0.5184,0.52,0.55,0.56,0.58,False,False,False,False,False,False,False,False


In [276]:
train_pred_no_limit_table.to_pickle("../data/train_pred_no_limit_table.pkl")
train_pred_2000_3000_table.to_pickle("../data/train_pred_2000_3000_table.pkl")
train_pred_3000_4000_table.to_pickle("../data/train_pred_3000_4000_table.pkl")

test_pred_no_limit_table.to_pickle("../data/test_pred_no_limit_table.pkl")
test_pred_2000_3000_table.to_pickle("../data/test_pred_2000_3000_table.pkl")
test_pred_3000_4000_table.to_pickle("../data/test_pred_3000_4000_table.pkl")